In [ ]:
!pip install python-binance

In [ ]:
!pip install websocket-client

!pip install schedule

!pip install datetime

!pip install pytrends --user

In [ ]:
import websocket, json
import pandas as pd
import numpy as np
import datetime
import csv
import schedule
import time
from binance.client import Client
from binance.enums import *
from binance.exceptions import BinanceAPIException, BinanceOrderException
from binance.enums import ORDER_TYPE_MARKET
import requests
from matplotlib import pyplot as plt
from itertools import accumulate
from datetime import datetime
import math # yukarı yuvarlama = math.ceil(x)
from pytrends.request import TrendReq
from scipy.stats import pearsonr

In [ ]:
futures_websocket = 'wss://fstream3.binance.com/ws/sandusdt@kline_5m'
client = Client('5NGfSQgOxmGMPH89lDOEkKNiWWxfSJDDDE2wz5n8hb2Wm9qy0oDmrxP0ZQVnFrRj' , 'wODRJqaFmiuZkyNW466xKCNYNRqDw6i8QVJJHwWW9GrFNl1lpwivdNZ8FqAYLMSo')

In [ ]:
TRADE_SYMBOL = 'SANDUSDT'

In [ ]:
data = client.futures_klines(symbol=TRADE_SYMBOL, interval = Client.KLINE_INTERVAL_5MINUTE,limit=102)
dff = pd.DataFrame(data)
dff_edited = dff.drop([0,1,2,3,5,6,7,8,9,10,11], axis=1)
df = dff_edited.drop(dff_edited.tail(1).index)
df.columns = ['c']
df["c"] = pd.to_numeric(df["c"], downcast="float")
closes =[]
zaman =[]
datam = [] 
mum=[]
blc=[]
t=[]
h=[]
l=[]
long_ins=[10]           
short_ins=[10]
def order(side, quantity, symbol, order_type=ORDER_TYPE_MARKET):
  try:
    print('sending order')
    order = client.futures_create_order(
              symbol=TRADE_SYMBOL,
              side=side,
              type=order_type,
              quantity=quantity)
    print('order ok')
  except Exception as e:
    return False
  return True

### client.futures_create_order(symbol=sym, side='BUY', type='MARKET', quantity=quantity) ###
longy=False
shorty=False
hunt=False
farm=False
#retreat=True
def getdata(symbol,interval,lookback):
    frame= pd.DataFrame(client.futures_historical_klines(symbol,interval,lookback))
    frame= frame.iloc[:,:6]
    frame.columns = ['Time','Open','High','Low','Close','Volume']
    frame= frame.astype(float)
    return frame
def on_open(ws):
  print('Receiving Data...')
def on_close(ws):
  print('Connection Closed')
def on_message(ws, message):
    global df
    global closes
    global zaman
    global datam
    global mum  
    global h
    global l
    global shorty
    global longy
    global hunt
    global farm
    #global retreat
    global getdata
    global long_ins
    global short_ins
    json_message = json.loads(message)
    #print(json_message)
    candle = json_message['k']
    candle_closed = candle['x']
    unix_time = candle['T']
    close_data = candle['c']
    high=candle['h']
    low=candle['l']
    df["x"] = pd.to_numeric(candle_closed, downcast="float")
    df = df.loc[df['x']==1]
    veri=[unix_time, close_data,candle_closed]
    veri1 = np.array([unix_time])
    veri2 = np.array([close_data])
    veri3 = np.array([candle_closed])
    veri4= np.array([high])
    veri5= np.array([low])
    #print(veri2)
    closes.append(float(veri2))
    zaman.append(float(veri1))
    mum.append(float(veri3))
    h.append(float(veri4))
    l.append(float(veri5))
    df = pd.DataFrame(closes,zaman)
    df.reset_index(inplace=True)
    df = df.rename(columns = {'index':'Time'})
    df = df.rename(columns = {0:'Close'})
    df['Mum']=mum
    df['High']=df['Close'].rolling(window=9000,min_periods=1).max().fillna(0)
    df['Low']=df['Close'].rolling(window=9000,min_periods=1).min().fillna(0)
    df['H']= h
    df['L']= l
    df=df.tail(36000)
    df.reset_index(inplace=True)
    row_index = df.Mum == 1
    df5=pd.DataFrame()
    df5['Close']=df.loc[row_index, 'Close']
    df5['High_End']= df.loc[row_index, 'H']
    df5['Low_End']= df.loc[row_index,'L']
    df5['Açıklık']= df5['High_End']-df5['Low_End']
    df5['Mean_Açıklık']= df5['Açıklık'].rolling(window=30,min_periods=1).mean().fillna(0)
    df5['Std_Açıklık']= df5['Açıklık'].rolling(window=30,min_periods=1).std().fillna(0)
    df5['Range']= df5['Mean_Açıklık']+df5['Std_Açıklık']
    rangy=df5['Range'].iloc[-1]
    df5.reset_index(inplace=True)
    price=df['Close'].iloc[-1]
    maxy=df['High'].iloc[-1]
    miny=df['Low'].iloc[-1]
    timy=df['Time'].iloc[-1]
    xlong=maxy-rangy
    xshort=miny+rangy
    QTY=2
    if len(df5.index)>12:
        #if retreat:
        if veri3:
            asset='SANDUSDT'
            asset2='BTCUSDT'
            dfq =getdata(asset, '5m', '1440m')
            dr =getdata(asset2, '5m', '1440m')
            dfq['Volatility']=dfq['Close'].rolling(window=6,min_periods=1).std().fillna(0)
            dfq['Vol_MA']=dfq['Volatility'].rolling(window=10).mean()
            volat= np.where(dfq['Vol_MA'].iloc[-1]>0.02,1,0)
            pytrends= TrendReq()
            keywords=['Bitcoin']
            pytrends.build_payload(keywords,timeframe='now 1-d')
            dataw=pytrends.interest_over_time()
            dataw['Tr']=dataw['Bitcoin'].rolling(window=5,min_periods=1).max().fillna(0)
            trend= np.where(dataw['Tr'].iloc[-1]>90,1,0)
            dfcorr=pd.DataFrame()
            dfcorr['corr']=dr['Close'].rolling(30).corr(dfq['Close'])
            c=dfcorr['corr'].abs()
            corr= np.where(c.iloc[-1]>0.80,1,0)
            volume=dfq['Volume'].rolling(window=10).mean()/1000000
            volu= np.where(volume.iloc[-1]>1,1,0)
            variable=volu+(trend*corr)+volat
            if variable>1:
                hunt=True
                print('Hunt Mode')
            else:
                farm=True
                print('Farm Mode')
        else:            
            if hunt:
                if price<=miny:
                    if longy:
                        order(SIDE_SELL,QTY,TRADE_SYMBOL)
                        if order:
                            longy=False
                            print('...xlongy...')                        
                            dfpos = pd.DataFrame({"Time":[timy],"Price":[price],"Mode":['H'],"Way":['L'],"In-Out":['OUT']})
                            dfpos.to_csv('Position.csv',header=False)
                    if not shorty:
                        order(SIDE_SELL,QTY,TRADE_SYMBOL)
                        if order:
                            shorty=True
                            print('...shorty...')
                            dfpos = pd.DataFrame({"Time":[timy],"Price":[price],"Mode":['H'],"Way":['S'],"In-Out":['IN']})
                            dfpos.to_csv('Position.csv',header=False)
                if shorty:
                    if price>=xshort:
                        order(SIDE_BUY,QTY,TRADE_SYMBOL)
                        if order:
                            shorty=False
                            print('...xshorty...')
                            hunt=False
                            dfpos = pd.DataFrame({"Time":[timy],"Price":[price],"Mode":['H'],"Way":['S'],"In-Out":['OUT']})
                            dfpos.to_csv('Position.csv',header=False)
                if price>=maxy:
                    if shorty:
                        order(SIDE_BUY,QTY,TRADE_SYMBOL)
                        if order:
                            shorty=False
                            print('...xshorty...')  
                            dfpos = pd.DataFrame({"Time":[timy],"Price":[price],"Mode":['H'],"Way":['S'],"In-Out":['OUT']})
                            dfpos.to_csv('Position.csv',header=False)
                    if not longy:
                        order(SIDE_BUY,QTY,TRADE_SYMBOL)
                        if order:
                            longy=True
                            print('...longy...')
                            dfpos = pd.DataFrame({"Time":[timy],"Price":[price],"Mode":['H'],"Way":['L'],"In-Out":['IN']})
                            dfpos.to_csv('Position.csv',header=False)
                if longy:
                    if price<=xlong:
                        order(SIDE_SELL,QTY,TRADE_SYMBOL)
                        if order:
                            longy=False
                            print('...xlongy...')
                            hunt=False
                            dfpos = pd.DataFrame({"Time":[timy],"Price":[price],"Mode":['H'],"Way":['L'],"In-Out":['OUT']})
                            dfpos.to_csv('Position.csv',header=False)
                

            if farm:
                if price<=miny:
                    if shorty:
                        order(SIDE_BUY,QTY,TRADE_SYMBOL)
                        if order:
                            shorty=False
                            print('...xshorty...')
                            dfpos = pd.DataFrame({"Time":[timy],"Price":[price],"Mode":['F'],"Way":['S'],"In-Out":['OUT']})
                            dfpos.to_csv('Position.csv',header=False)
                    if not longy:
                        order(SIDE_BUY,QTY,TRADE_SYMBOL)
                        if order:    
                            long_ins.append(price)    
                            longy=True
                            print('...longy...')
                            dfpos = pd.DataFrame({"Time":[timy],"Price":[price],"Mode":['F'],"Way":['L'],"In-Out":['IN']})
                            dfpos.to_csv('Position.csv',header=False)
                if shorty:
                    short_out=short_ins[-1]*1.01 
                    if price<=xlong or price>short_out:
                        order(SIDE_BUY,QTY,TRADE_SYMBOL)
                        if order:
                            shorty=False
                            print('...xshorty...')
                            farm=False
                            dfpos = pd.DataFrame({"Time":[timy],"Price":[price],"Mode":['F'],"Way":['S'],"In-Out":['OUT']})
                            dfpos.to_csv('Position.csv',header=False)
                            while veri3==0:
                                continue
                            else:
                                break
                else:
                    break
                    
                if price>=maxy:
                    if longy:
                        order(SIDE_SELL,QTY,TRADE_SYMBOL)
                        if order:
                            longy=False
                            print('...xlongy...')
                            dfpos = pd.DataFrame({"Time":[timy],"Price":[price],"Mode":['F'],"Way":['L'],"In-Out":['OUT']})
                            dfpos.to_csv('Position.csv',header=False)
                    if not shorty:
                        order(SIDE_SELL,QTY,TRADE_SYMBOL)
                        if order:
                            short_ins.append(price) 
                            shorty=True
                            print('...shorty...')
                            dfpos = pd.DataFrame({"Time":[timy],"Price":[price],"Mode":['F'],"Way":['S'],"In-Out":['IN']})
                            dfpos.to_csv('Position.csv',header=False)
                if longy:
                    long_out=long_ins[-1]*0.99
                    if price>=xshort or price<long_out:
                        order(SIDE_SELL,QTY,TRADE_SYMBOL)
                        if order:
                            longy=False
                            print('...xlongy...')
                            farm=False        
                            dfpos = pd.DataFrame({"Time":[timy],"Price":[price],"Mode":['F'],"Way":['L'],"In-Out":['OUT']})
                            dfpos.to_csv('Position.csv',header=False)
                            while veri3==0:
                                continue
                            else:
                                break
                       
            
    balances=client.futures_account_balance()
    balance=balances[6]['balance']
    time=balances[8]['updateTime']
    global blc
    global t
    blc.append(float(balance))
    t.append(float(time))
    dfg = pd.DataFrame(blc,t)
    dfg.reset_index(inplace=True)
    dfg = dfg.rename(columns = {'index':'Time'})
    dfg = dfg.rename(columns = {0:'Balance'})
    dfg = dfg.drop_duplicates(keep='last')
    dfg.to_csv('fightorflight.csv',mode='w+' )
    

    
    
    
ws = websocket.WebSocketApp(futures_websocket, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()

In [ ]:
24.01.22 20.21